In [1]:
import numpy as np
from qiskit import (QuantumCircuit,execute,Aer)
from qiskit.visualization import plot_histogram
from qiskit import*
from django.db import models
from docplex.mp.model import Model                                                                                                      
import docplex
# import qiskit.optimization.ising.tsp.TspData as tp
from qiskit.optimization.ising.tsp import TspData
# from qiskit.aqua.optimization import TspData
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.ising import tsp
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.algorithms import QAOA, ExactEigensolver

from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import COBYLA

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
from qiskit import IBMQ

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy

In [16]:
from qiskit import IBMQ
IBMQ.save_account()

In [ ]:
IBMQ.load_account()

In [32]:
def data(p):
    p = "gr17.tsp.txt"
    anyway = open(p , "r")
    faint = anyway.read().split()

    a = []
    for i in range(len(faint)):
        a.append(int(faint[i]))

    b = np.zeros((17,17))
    t = []
    for i in range(len(a)):
        if a[i] != 0:
            t.append(i)
    counts=0
    for k in range(len(b) - 1):
        for l in range(k+1 , len(b)):
            b[k][l] = a[t[counts]]
            counts+=1


    G = b+ b.T
    return G
p = "gr17.tsp.txt"



In [33]:
def randomM(rand):
    G = data(p)
    rand = 3
    new = np.random.randint(rand, len(G) - rand)

    A = G[new:rand+new,new:rand+new]
    return A
randomM(3)

array([[  0., 240., 140.],
       [240.,   0., 237.],
       [140., 237.,   0.]])

In [22]:
def vqealgo(A):
    x = TspData('tmp',len(A),np.zeros((3,3)),A)
    qubitOp, offset = tsp.get_operator(x)

    seed = 10598
    spsa = SPSA(max_trials = 300)
    ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
    vqe = VQE(qubitOp, ry, spsa)

    my_provider = IBMQ.get_provider('ibm-q-wits')
    my_provider.backends()
    device = my_provider.get_backend('ibmq_16_melbourne')

    quantum_instance = QuantumInstance(device, seed_simulator=seed, seed_transpiler=seed , skip_qobj_validation = False,shots = 100 )

    result = vqe.run(quantum_instance)
    w = sample_most_likely(result['eigvecs'][0])
    y =  tsp.tsp_feasible(w)
    t =   result['eval_time']

    z = tsp.get_tsp_solution(w)
    p =  tsp.tsp_value(z, A)
 
    return (t , y , p)
vqealgo(A)